In [12]:
# Load env variables and create client
from dotenv import load_dotenv
from anthropic import Anthropic

load_dotenv()

client = Anthropic()
model = "claude-haiku-4-5"

In [13]:
# Helper functions
def add_user_message(messages, text):
    user_message = {"role": "user", "content": text}
    messages.append(user_message)


def add_assistant_message(messages, text):
    assistant_message = {"role": "assistant", "content": text}
    messages.append(assistant_message)


def chat(messages, system=None, temperature=1.0, stop_sequences=[]):
    params = {
        "model": model,
        "max_tokens": 1000,
        "messages": messages,
        "temperature": temperature,
        "stop_sequences": stop_sequences,
    }

    if system:
        params["system"] = system

    message = client.messages.create(**params)
    return message.content[0].text

In [55]:
import json


def generate_dataset():
    prompt = """
Generate a evaluation dataset for a prompt evaluation. The dataset will be used to evaluate prompts
that generate Python, JSON, or Regex specifically for AWS-related tasks. Generate an array of JSON objects,
each representing task that requires Python, JSON, or a Regex to complete.

Example output:
```json
[
    {
        "task": "Description of task",
        "format": "json" or "python" or "regex",
        "solution_criteria": "Must include runtime, memory size, timeout and basic structure for AWS Lambda configuration"
    },
    ...additional
]
```

* Focus on tasks that can be solved by writing a single Python function, a single JSON object, or a regular expression.
* Focus on tasks that do not require writing much code

Please generate 3 objects.
"""

    messages = []
    add_user_message(messages, prompt)
    add_assistant_message(messages, "```json")
    text = chat(messages, stop_sequences=["```"])
    return json.loads(text)

In [56]:
dataset = generate_dataset()

# Commenting this out so as not to overwrite generated dataset
with open("aws_prompt_evaluation_dataset.json", "w") as f:
    json.dump(dataset, f, indent=2)

In [57]:
def run_prompt(test_case):
    """Merges the prompt and test case input, then returns the result"""
    prompt = f"""
Please solve the following task:

{test_case["task"]}

* Respond only with Python, JSON, or a plain Regex
* Do not add any comments or commentary or explanation
"""

    messages = []
    add_user_message(messages, prompt)
    add_assistant_message(messages, "```code")
    output = chat(messages, stop_sequences=["```"])
    return output

In [59]:
def grade_by_model(test_case, output):
    eval_prompt = f"""
You are an expert AWS code reviewer. Your task is to evaluate the following AI-generated solution.

Original Task:
<task>
{test_case["task"]}
</task>

Solution to Evaluate:
<solution>
{output}
</solution>

Solution criteria to Evaluate:
<solution_criteria>
{test_case["solution_criteria"]}
</solution_criteria>

Output Format
Provide your evaluation as a structured JSON object with the following fields, in this specific order:
- "strengths": An array of 1-3 key strengths
- "weaknesses": An array of 1-3 key areas for improvement
- "reasoning": A concise explanation of your overall assessment
- "score": A number between 1-10

Respond with JSON. Keep your response concise and direct.
Example response shape:
{{
    "strengths": string[],
    "weaknesses": string[],
    "reasoning": string,
    "score": number
}}
    """

    messages = []
    add_user_message(messages, eval_prompt)
    add_assistant_message(messages, "```json")
    eval_text = chat(messages, stop_sequences=["```"])
    return json.loads(eval_text)

In [42]:
# Functions to validate the output structure
import re
import ast


def validate_json(text):
    try:
        json.loads(text.strip())
        return 10
    except json.JSONDecodeError:
        return 0


def validate_python(text):
    try:
        ast.parse(text.strip())
        return 10
    except SyntaxError:
        return 0


def validate_regex(text):
    try:
        re.compile(text.strip())
        return 10
    except re.error:
        return 0


def grade_syntax(response, test_case):
    format = test_case["format"]
    if format == "json":
        return validate_json(response)
    elif format == "python":
        return validate_python(response)
    else:
        return validate_regex(response)


In [51]:
def run_test_case(test_case):
  """Calls run_prompt, then grades the result"""
  output = run_prompt(test_case)

  model_grade = grade_by_model(test_case, output)
  model_score = model_grade["score"]
  reasoning = model_grade["reasoning"]

  syntax_score = grade_syntax(output, test_case)
  score = (syntax_score + model_score) / 2

  return {
    "output": output,
    "test_case": test_case,
    "score": score,
    "model_score": model_score,
    "syntax_score": syntax_score,
    "reasoning": reasoning
  }

In [60]:
from statistics import mean

def run_eval(dataset):
    """Loads the dataset and calls run_test_case with each case"""
    results = []
    for test_case in dataset:
        result = run_test_case(test_case)
        results.append(result)

    average_score = mean([result["score"] for result in results])
    print(f"Average score: {average_score}")

    return results

In [61]:
with open("aws_prompt_evaluation_dataset.json", "r") as f:
    dataset = json.load(f)

results = run_eval(dataset)

Average score: 6.166666666666667


In [62]:
print(json.dumps(results, indent=2))

[
  {
    "output": "\nimport json\n\ntemplate = {\n    \"AWSTemplateFormatVersion\": \"2010-09-09\",\n    \"Description\": \"CloudFormation template for S3 bucket with versioning enabled and public access blocked\",\n    \"Resources\": {\n        \"S3Bucket\": {\n            \"Type\": \"AWS::S3::Bucket\",\n            \"Properties\": {\n                \"BucketName\": {\n                    \"Fn::Sub\": \"my-bucket-${AWS::AccountId}-${AWS::Region}\"\n                },\n                \"VersioningConfiguration\": {\n                    \"Status\": \"Enabled\"\n                },\n                \"PublicAccessBlockConfiguration\": {\n                    \"BlockPublicAcls\": True,\n                    \"BlockPublicPolicy\": True,\n                    \"IgnorePublicAcls\": True,\n                    \"RestrictPublicBuckets\": True\n                }\n            }\n        }\n    },\n    \"Outputs\": {\n        \"BucketName\": {\n            \"Description\": \"Name of the S3 bucket\",\